In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from geopandas import GeoDataFrame
from sklearn import preprocessing

plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.labelsize'] = 12

#### No estoy seguro si esta bien, pero pruebo

In [2]:
propiedades = pd.read_csv('properati.csv')

In [3]:
def obtener_barrio(fila):
    barrio,padre = fila
    return padre.split('|')[3]

In [4]:
propiedades.loc[:, 'place_name'] = propiedades.loc[:, ['place_name','place_with_parent_names']].apply(lambda x: obtener_barrio(x), axis = 1)

In [5]:
tipo_propiedad = {'PH': [0], 'apartment':[587], 'house': [1829], 'store': [28531]}

In [6]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipo_propiedad[x]) 

In [7]:
import random
barrios = {}
x = 0
for i in propiedades.groupby('place_name').agg(np.mean).lat.keys():
    if i in barrios:
        continue
    barrios[i] = [x]
    x+=random.randint(10, 100)

In [8]:
propiedades.reset_index(drop = True, inplace = True)

In [9]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: barrios[x]) 

In [10]:
from sklearn import preprocessing
tipos = preprocessing.OneHotEncoder()
tipos.fit([[0], [587], [1829], [28531]]) 

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [11]:
preparacion = barrios.values()
for i in range (0, len(barrios)):
    preparacion[i] = preparacion[i]

In [12]:
pre_barrios = preprocessing.OneHotEncoder()
pre_barrios.fit(preparacion)

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [13]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: pre_barrios.transform([x]))

In [14]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipos.transform([x]))

In [15]:
adivinar = propiedades.loc[propiedades.mes >= 7,:]

In [16]:
entrenar = propiedades.loc[propiedades.mes < 7,:]

In [17]:
condiciones = []
precios = []

for i in range(0,len(entrenar)):
    if np.isnan(entrenar.price_aprox_usd[i]):
        continue
    if np.isinf(entrenar.ditancia_subtes[i]):
        continue
    l = list(entrenar.place_name[i].toarray()[0]) + list(entrenar.property_type[i].toarray()[0])
    l.append(entrenar.ditancia_subtes[i])
    l.append(entrenar.seguridad[i])
    l.append(entrenar.gimnasio[i])
    condiciones.append(l)
    precios.append(entrenar.price_aprox_usd[i])

In [18]:
def adivinar_precio_indice(fila, estimador):
    barrio,tipo,subte, seg, gim = fila
    l =  list(barrio.toarray()[0]) + list(tipo.toarray()[0])
    if np.isinf(subte):
        return 0 # Ver de resolver estos despues
    l.append(subte)
    l.append(seg)
    l.append(gim)
    return  estimador.predict([l])[0]

## Con Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


In [79]:
#propiedades.place_name[0].toarray()[0]
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.9,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [81]:
adivinar.loc[:,'precio_adivinado_gradient'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, est), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [82]:
adivinar.loc[:,'error_porcentual_gradient'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [106]:
adivinar.loc[adivinar.precio_adivinado_gradient>1000,:].error_porcentual_gradient.describe()

count    16205.000000
mean        83.544287
std        201.700593
min          0.000000
25%         18.903710
50%         43.096393
75%         79.090711
max       9038.270013
Name: error_porcentual_gradient, dtype: float64

In [92]:
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [94]:
adivinar.loc[:,'precio_adivinado_gradient_01'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, est), axis = 1)

In [95]:
adivinar.loc[:,'error_porcentual_gradient_01'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient_01']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [105]:
adivinar.loc[adivinar.precio_adivinado_gradient_01>1000,:].error_porcentual_gradient_01.describe()

count    1.645800e+04
mean     6.942295e+01
std      1.631405e+02
min      8.623357e-14
25%      1.738087e+01
50%      3.798468e+01
75%      6.916166e+01
max      5.788523e+03
Name: error_porcentual_gradient_01, dtype: float64

In [108]:
est = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.9,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [109]:
adivinar.loc[:,'precio_adivinado_gradient_10000'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, est), axis = 1)

In [110]:
adivinar.loc[:,'error_porcentual_gradient_10000'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient_10000']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [112]:
adivinar.loc[adivinar.precio_adivinado_gradient_10000>1000,:].error_porcentual_gradient_10000.describe()

count    16181.000000
mean        83.795200
std        201.972879
min          0.000000
25%         18.898221
50%         43.257067
75%         79.274035
max       9033.411137
Name: error_porcentual_gradient_10000, dtype: float64

## Con SVR

In [25]:
from sklearn import svm

In [114]:
clf = svm.SVR(degree = 2)
clf.fit(condiciones, precios)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [116]:
adivinar.loc[:,'precio_adivinado_svr'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x,clf), axis = 1)

In [117]:
adivinar.loc[:,'error_porcentual_svr'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_svr']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [121]:
adivinar.loc[adivinar.precio_adivinado_svr != 0].error_porcentual_svr.describe()

count    16478.000000
mean        59.056386
std         96.720346
min          0.000235
25%         22.490996
50%         46.370952
75%         73.992968
max       2967.125531
Name: error_porcentual_svr, dtype: float64

In [26]:
# Le cambio el parametro degree y uso la funcion kernel. Creo que que si no esta seteado en poly
# degree no es usado y por lo tanto estaria innecesariamente 
clf = svm.SVR(kernel = 'poly', degree = 1)
clf.fit(condiciones, precios)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=1, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [29]:
adivinar.loc[:,'precio_adivinado_svr_1'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x,clf), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [30]:
adivinar.loc[:,'error_porcentual_svr_1'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_svr_1']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [37]:
adivinar.loc[adivinar.precio_adivinado_svr_1>1000].error_porcentual_svr_1.describe()

count    16478.000000
mean        59.146539
std         96.969947
min          0.006167
25%         22.453689
50%         46.100371
75%         73.841342
max       2991.126704
Name: error_porcentual_svr_1, dtype: float64

In [32]:
# Le cambio el parametro degree y uso la funcion kernel. Creo que que si no esta seteado en poly
# degree no es usado y por lo tanto estaria innecesariamente 
clf = svm.SVR(kernel = 'poly', degree = 2)
clf.fit(condiciones, precios)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [33]:
adivinar.loc[:,'precio_adivinado_svr_2'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x,clf), axis = 1)

In [34]:
adivinar.loc[:,'error_porcentual_svr_2'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_svr_2']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [36]:
adivinar.loc[adivinar.precio_adivinado_svr_2>1000,:].error_porcentual_svr_2.describe()

count    16478.000000
mean        59.053421
std         96.693109
min          0.000009
25%         22.449899
50%         46.421189
75%         74.103215
max       2967.543398
Name: error_porcentual_svr_2, dtype: float64

In [38]:
# Le cambio el parametro degree y uso la funcion kernel. Creo que que si no esta seteado en poly
# degree no es usado y por lo tanto estaria innecesariamente 
clf = svm.SVR(kernel = 'poly', degree = 3)
clf.fit(condiciones, precios)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [39]:
adivinar.loc[:,'precio_adivinado_svr_3'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x,clf), axis = 1)

In [40]:
adivinar.loc[:,'error_porcentual_svr_3'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_svr_3']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [41]:
adivinar.loc[adivinar.precio_adivinado_svr_3>1000,:].error_porcentual_svr_3.describe()

count    16478.000000
mean        59.163726
std         97.197111
min          0.000043
25%         22.472075
50%         46.247680
75%         74.074000
max       2974.914822
Name: error_porcentual_svr_3, dtype: float64

## Con arbol

In [19]:
from sklearn.tree import DecisionTreeRegressor

In [21]:
# Primero uso con profundidad 2
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_1.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [24]:
adivinar.loc[:,'precio_adivinado_arbol_2'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_1), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [25]:
adivinar.loc[:,'error_porcentual_arbol_2'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_2']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [26]:
adivinar.error_porcentual_arbol_2.describe()

count    39967.000000
mean        95.831373
std         94.024004
min          0.023175
25%         76.275653
50%        100.000000
75%        100.000000
max       3916.162386
Name: error_porcentual_arbol_2, dtype: float64

In [27]:
# Pruebo con profundidad 4
regr_2 = DecisionTreeRegressor(max_depth=4)
regr_2.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [29]:
adivinar.loc[:,'precio_adivinado_arbol_4'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_2), axis = 1)

In [30]:
adivinar.loc[:,'error_porcentual_arbol_4'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_4']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [31]:
adivinar.error_porcentual_arbol_4.describe()

count    39967.000000
mean        93.435456
std         96.417620
min          0.014512
25%         69.679316
50%        100.000000
75%        100.000000
max       7268.826253
Name: error_porcentual_arbol_4, dtype: float64

In [43]:
# Pruebo con profundidad 10
regr_3 = DecisionTreeRegressor(max_depth=10)
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [44]:
adivinar.loc[:,'precio_adivinado_arbol_10'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

In [45]:
adivinar.loc[:,'error_porcentual_arbol_10'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_10']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [46]:
adivinar.loc[adivinar.precio_adivinado_arbol_10>1000,:].error_porcentual_arbol_10.describe()

count    16478.000000
mean        74.957117
std        172.969022
min          0.000000
25%         21.207478
50%         44.572782
75%         82.618251
max       8166.666667
Name: error_porcentual_arbol_10, dtype: float64

In [20]:
# Pruebo con profundidad con mas muestras por hoja y modifico la funcion
#min_samples_leaf=30,
#criterion='mae'
regr_3 = DecisionTreeRegressor(max_depth=2, criterion='mae')
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mae', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [21]:
adivinar.loc[:,'precio_adivinado_arbol_mae'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
adivinar.loc[:,'error_porcentual_arbol_mae'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_mae']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [41]:
adivinar.loc[adivinar.precio_adivinado_arbol_mae>1000,:].error_porcentual_arbol_mae.describe()

count    16478.000000
mean        52.731373
std         91.319905
min          0.000000
25%         18.215613
50%         40.639269
75%         64.864865
max       2475.517134
Name: error_porcentual_arbol_mae, dtype: float64

In [24]:
regr_3 = DecisionTreeRegressor(max_depth=4, criterion='mae')
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mae', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [25]:
adivinar.loc[:,'precio_adivinado_arbol_mae_4'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

In [26]:
adivinar.loc[:,'error_porcentual_arbol_mae_4'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_mae_4']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [40]:
adivinar.loc[adivinar.precio_adivinado_arbol_mae_4>1000,:].error_porcentual_arbol_mae_4.describe()

count    16478.000000
mean        52.825290
std         98.109330
min          0.000000
25%         18.072289
50%         38.888889
75%         63.703704
max       4112.704341
Name: error_porcentual_arbol_mae_4, dtype: float64

In [28]:
regr_3 = DecisionTreeRegressor(max_depth=4, criterion='mae', min_samples_leaf=5)
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mae', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [30]:
adivinar.loc[:,'precio_adivinado_arbol_min_leaf_5'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

In [31]:
adivinar.loc[:,'error_porcentual_arbol_min_leaf_5'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_min_leaf_5']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [39]:
adivinar.loc[adivinar.precio_adivinado_arbol_min_leaf_5>1000,:].error_porcentual_arbol_min_leaf_5.describe()

count    16478.000000
mean        52.825290
std         98.109330
min          0.000000
25%         18.072289
50%         38.888889
75%         63.703704
max       4112.704341
Name: error_porcentual_arbol_min_leaf_5, dtype: float64

In [33]:
regr_3 = DecisionTreeRegressor(max_depth=4, criterion='mae', min_samples_leaf=15)
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mae', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=15,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [34]:
adivinar.loc[:,'precio_adivinado_arbol_min_leaf_15'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

In [35]:
adivinar.loc[:,'error_porcentual_arbol_min_leaf_15'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_min_leaf_15']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [38]:
adivinar.loc[adivinar.precio_adivinado_arbol_min_leaf_15>1000,:].error_porcentual_arbol_min_leaf_15.describe()

count    16478.000000
mean        52.825290
std         98.109330
min          0.000000
25%         18.072289
50%         38.888889
75%         63.703704
max       4112.704341
Name: error_porcentual_arbol_min_leaf_15, dtype: float64

In [47]:
regr_3 = DecisionTreeRegressor(max_depth=4, criterion='mae', min_samples_leaf=30)
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mae', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=30,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [48]:
adivinar.loc[:,'precio_adivinado_arbol_min_leaf_30'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

In [49]:
adivinar.loc[:,'error_porcentual_arbol_min_leaf_30'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_min_leaf_30']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [50]:
adivinar.loc[adivinar.precio_adivinado_arbol_min_leaf_30>1000,:].error_porcentual_arbol_min_leaf_30.describe()

count    16478.000000
mean        52.872060
std         98.293727
min          0.000000
25%         17.993626
50%         38.888889
75%         63.703704
max       4112.704341
Name: error_porcentual_arbol_min_leaf_30, dtype: float64

In [51]:
regr_3 = DecisionTreeRegressor(max_depth=4, criterion='mae', min_samples_leaf=10, splitter='random')
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mae', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='random')

In [52]:
adivinar.loc[:,'precio_adivinado_arbol_random'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, regr_3), axis = 1)

In [53]:
adivinar.loc[:,'error_porcentual_arbol_random'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_random']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [54]:
adivinar.loc[adivinar.precio_adivinado_arbol_random>1000,:].error_porcentual_arbol_random.describe()

count    16478.000000
mean        53.177076
std         96.255988
min          0.000000
25%         18.147448
50%         39.130435
75%         64.182312
max       4112.704341
Name: error_porcentual_arbol_random, dtype: float64